#### **Utils**

In [1]:
import matplotlib.pyplot as plt
import numpy as np


def add_intercept(x):
    """Add intercept to matrix x.

    Args:
        x: 2D NumPy array.

    Returns:
        New matrix same as x with 1's in the 0th column.
    """
    new_x = np.zeros((x.shape[0], x.shape[1] + 1), dtype=x.dtype)
    new_x[:, 0] = 1
    new_x[:, 1:] = x

    return new_x


def load_dataset(csv_path, label_col='y', add_intercept=False):
    """Load dataset from a CSV file.

    Args:
         csv_path: Path to CSV file containing dataset.
         label_col: Name of column to use as labels (should be 'y' or 'l').
         add_intercept: Add an intercept entry to x-values.

    Returns:
        xs: Numpy array of x-values (inputs).
        ys: Numpy array of y-values (labels).
    """

    def add_intercept_fn(x):
        global add_intercept
        return add_intercept(x)

    # Validate label_col argument
    allowed_label_cols = ('y', 't')
    if label_col not in allowed_label_cols:
        raise ValueError('Invalid label_col: {} (expected {})'
                         .format(label_col, allowed_label_cols))

    # Load headers
    with open(csv_path, 'r') as csv_fh:
        headers = csv_fh.readline().strip().split(',')

    # Load features and labels
    x_cols = [i for i in range(len(headers)) if headers[i].startswith('x')]
    l_cols = [i for i in range(len(headers)) if headers[i] == label_col]
    inputs = np.loadtxt(csv_path, delimiter=',', skiprows=1, usecols=x_cols)
    labels = np.loadtxt(csv_path, delimiter=',', skiprows=1, usecols=l_cols)

    if inputs.ndim == 1:
        inputs = np.expand_dims(inputs, -1)

    if add_intercept:
        inputs = add_intercept_fn(inputs)

    return inputs, labels


def plot(x, y, theta, save_path=None, correction=1.0):
    """Plot dataset and fitted logistic regression parameters.
    Args:
        x: Matrix of training examples, one per row.
        y: Vector of labels in {0, 1}.
        theta: Vector of parameters for logistic regression model.
        save_path: Path to save the plot.
        correction: Correction factor to apply (Problem 2(e) only).
    """
    # Plot dataset
    plt.figure()
    plt.plot(x[y == 1, -2], x[y == 1, -1], 'bx', linewidth=2)
    plt.plot(x[y == 0, -2], x[y == 0, -1], 'go', linewidth=2)

    # Plot decision boundary (found by solving for theta^T x = 0)
    margin1 = (max(x[:, -2]) - min(x[:, -2]))*0.2
    margin2 = (max(x[:, -1]) - min(x[:, -1]))*0.2
    x1 = np.arange(min(x[:, -2])-margin1, max(x[:, -2])+margin1, 0.01)
    x2 = -(theta[0] / theta[2] * correction + theta[1] / theta[2] * x1)
    plt.plot(x1, x2, c='red', linewidth=2)
    plt.xlim(x[:, -2].min()-margin1, x[:, -2].max()+margin1)
    plt.ylim(x[:, -1].min()-margin2, x[:, -1].max()+margin2)

    # Add labels and save to disk
    plt.xlabel('x1')
    plt.ylabel('x2')
    if save_path is not None:
        plt.savefig(save_path)


# **Linear Regression**

---

## **Loss (Cost) Function**

$$
J(\boldsymbol{\theta}) = \frac{1}{2m} 
\sum_{i=1}^{m} \left( h_{\boldsymbol{\theta}}(\mathbf{x}^{(i)}) - y^{(i)} \right)^2
$$

---

## **Hypothesis Function**

$$
h_{\boldsymbol{\theta}}(\mathbf{x}^{(i)}) = 
\boldsymbol{\theta}^\top \mathbf{x}^{(i)}
$$

---

## **Notation Summary**

| Symbol | Meaning |
|:--:|:--|
| $m$ | number of training examples |
| $\mathbf{x}^{(i)}$ | input feature vector of the $i^{th}$ example |
| $y^{(i)}$ | true label for the $i^{th}$ example |
| $\boldsymbol{\theta}$ | parameter (weight) vector |
| $h_{\boldsymbol{\theta}}(x)$ | predicted value (hypothesis) |

---

💡 *Goal:* Find parameters $\boldsymbol{\theta}$ that minimize the cost function $J(\boldsymbol{\theta})$, i.e.  
$$
\boldsymbol{\theta}^* = \arg\min_{\boldsymbol{\theta}} J(\boldsymbol{\theta})
$$


In [3]:
class LinearModel(object):
    """Base class for linear models"""
    def __init__(self, step_size, max_iter=100, eps=1e-6,
                theta_0=None, verbose=True):
        
        self.theta = theta_0
        self.step_size = step_size
        self.max_iter = max_iter
        self.eps = eps
        self.verbose = verbose

    def fit(X, y):
        """Run solver so fit linear model.
        
        Args:
            X: Training example inputs. Shape (m, n)
            y: Training example labels. Shape (m, )
        """

        raise NotImplementedError("Subclass of LinearModel must implement fit method.")

    def predict(self, X):

        """Make a pediction given new inputs x
        Args:
            X: Input of shape(m, n)
        
        Returns:
            Output of shape(m, )"""

        raise NotImplemetedError("Subclass of LinearModel must implement predict method.")
        
    

# **1. Batch Gradient Descent(BGD)**

In [7]:
class LinearRegessionBatchGradientDescent(LinearModel):
    def fit(self, X, y):
        m, n = X.shape

        # Initialize parameters
        self.theta = np.zeros(n)

        for _ in range(self.max_iter):
            theta_old = np.copy(self.theta)
            
            # Compute gradient
            gradient = (1/(2*m)) * (X @ self.theta - y) @ X
            
            # Update parameters
            self.theta -= self.learning_rate * gradient
            
            # Check convergence
            if np.linalg.norm(self.theta - theta_old, ord=1) < self.eps:
                break

    def predict(self, X):
        return X @ self.theta
        
def main():
    pass

# **2. Stochatic Gradient Descent(BGD)**

In [8]:
class LinearRegressionStochaticGradientDescent(LinearModel):

    def fit(self, X, y):
        m, n = X.shape()

        #Initialize parameters
        self.theta = np.zeros(n)
        
        for epoch in range(self.epochs):
            theta_old = np.copy(self.theta)

            # Shuffle data each epoch
            indices = np.random.permutation(m)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            # Update weights per sample
            for i in range(m):
                xi = X_shuffled[i:i+1]          # shape (1, n)
                yi = y_shuffled[i]
                gradient = 1/2 * (xi @ self.theta - yi) @ xi
                self.theta -= self.learning_rate * gradient.flatten()

            # Early stopping if change small
            if np.linalg.norm(self.theta - theta_old, ord=1) < self.tol:
                break

    def predict(self, X):
        return X @ self.theta        

def main():
    pass        

# **3. Normal Equation**

In [6]:
class LinearRegressionNormalEquation(LinearModel):
    def fit(self, X, y):
        # Normal equation: θ = (XᵀX)⁻¹ Xᵀy
        self.theta = np.linalg.pinv(X.T @ X) @ X.T @ y

    def predict(self, X):
        return X @ self.theta
def main():
    pass

# **4. Locally Weighted Linear Regression**

In [9]:
class LocallyWeightedLinearRegression:
    def __init__(self, tau):
        self.tau = tau
        self.x = None
        self.y = None

    def fit(self, X, y):
        """Store training data."""
        self.x = X
        self.y = y

    def predict(self, X):
        """Make predictions given inputs X.
        
        Args:
            X: shape (m, n)
        Returns:
            y_pred: shape (m,)
        """
        m, n = X.shape
        y_pred = np.zeros(m)

        for i in range(m):
            # compute weights for each training sample relative to query X[i]
            diff = self.x - X[i]                     # (m, n)
            W = np.diag(np.exp(-np.sum(diff**2, axis=1) / (2 * self.tau**2)))

            # solve for theta = (X^T W X)^(-1) X^T W y
            A = self.x.T @ W @ self.x
            b = self.x.T @ W @ self.y
            # regularization for stability
            theta = np.linalg.solve(A + 1e-5 * np.eye(n), b)

            # predict y_hat = x_i^T theta
            y_pred[i] = X[i] @ theta

        return y_pred
